In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/Yes_Bank_Train.csv")

In [ ]:
df.dropna(inplace=True)

In [ ]:
k = df["credit_amount"].values
df.drop(["credit_amount"], axis=1, inplace=True)

In [ ]:
y = []
for n in range(len(k)):
    if k[n]<20000 and k[n]>4000:
        y.append([1,0,0])
    elif k[n]<4000 and k[n]>1500:
        y.append([0,1,0])
    elif k[n]<1500:
        y.append([0,0,1])
y = np.array(y)

In [ ]:
obj =  ["account_info", "credit_history", "purpose", "savings_account", "employment_st", "personal_status", "gurantors", "property_type", "installment_type", "housing_type","job_type", "telephone", "foreigner"]
df = pd.get_dummies(df,columns=obj)
#df.drop(obj,axis=1,inplace=True)

In [ ]:
df.drop("serial number", axis=1,inplace=True)

In [ ]:
x_train = df.iloc[:640,:].values
x_test = df.iloc[640:800,:].values
y_train = y[:640,:]
y_test = y[640:800,:]

In [ ]:
print(x_train.shape,y_train.shape)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Activation, Input, Add

In [ ]:
#model 1
hp = 0.2
inputs = Input(shape=[60])
x = Dense(1024, input_dim = [60])(inputs)
x = Dropout(hp)(x)

x_sh = x
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)

x = Dense(256)(x)
x_sh = x
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(128)(x)
x_sh = x
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#x = Dropout(hp)(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)
pre = Dense(3, activation='softmax')(x)
model_4 = Model(input=inputs, output=pre)
model_4.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_4.summary()

In [ ]:
model_4.fit(x_train, y_train,validation_data=(x_test,y_test), batch_size=1024, epochs=1000)

In [ ]:
df2 = pd.read_csv("../input/Yes_Bank_Test_int.csv")

In [ ]:
df2 = pd.get_dummies(df2,columns=obj)
s_no = df2["serial number"].values
df2.drop("serial number", axis=1,inplace=True)

In [ ]:
test_data = df2.values

In [ ]:
preds = model_4.predict(test_data)
ans = []
for l in preds:
    for n in range(len(l)):
        if l[n]==max(l):
            ans.append(n+1)
data ={"serial number": s_no, "cluster_number": ans }
df3 = pd.DataFrame(data=data)
df3.to_csv("submission.csv")

In [ ]:
#model 3
hp = 0.25
inputs = Input(shape=[60])
x = Dense(1024, input_shape = [60])(inputs)
x = Dropout(hp)(x)

x_sh = x
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
#x = Dropout(hp)(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)

x = Dense(256)(x)
x_sh = x
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
#x = Dropout(hp)(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)

pre = Dense(3, activation='softmax')(x)
model_3 = Model(input=inputs, output=pre)
model_3.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model_3.fit(x_train, y_train,validation_data=(x_test,y_test), batch_size=1024, epochs=1000)

In [ ]:
#model 1
hp = 0.2
inputs = Input(shape=[60])
x = Dense(1024, input_dim = [60])(inputs)
x = Dropout(hp)(x)

x_sh = x
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)

x = Dense(256)(x)
x_sh = x
x = Dense(256)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(128)(x)
x_sh = x
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(64)(x)
x_sh = x
x = Dense(64)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dropout(hp)(x)
x = Add()([x, x_sh])
x = Activation('relu')(x)
pre = Dense(3, activation='softmax')(x)
model_5 = Model(input=inputs, output=pre)
model_5.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_5.fit(x_train, y_train,validation_data=(x_test,y_test), batch_size=1024, epochs=1000)

In [ ]:
preds2 = model_5.predict(test_data)
ans = []
for l in preds2:
    for n in range(len(l)):
        if l[n]==max(l):
            ans.append(n+1)
data ={"serial number": s_no, "cluster_number": ans }
df3 = pd.DataFrame(data=data)
df3.to_csv("submission_3.csv")

In [ ]:
#model 12
model_12 = Sequential()

drpt_val=0.75

model_12.add(Dense(256, input_shape = [60]))
model_12.add(Dropout(0.25))
model_12.add(Activation('tanh'))

model_12.add(Dense(1024))
model_12.add(BatchNormalization())
model_12.add(Dropout(drpt_val))

#model_2.add(Dense(1024))
#model_2.add(BatchNormalization())
#model_2.add(Dropout(0.25))

model_12.add(Dense(512))
model_12.add(BatchNormalization())
model_12.add(Dropout(drpt_val))

model_12.add(Dense(256))
model_12.add(BatchNormalization())
model_12.add(Dropout(drpt_val))

model_12.add(Dense(128))
model_12.add(BatchNormalization())
model_12.add(Dropout(drpt_val))

model_12.add(Dense(3,activation='softmax'))
model_12.compile(optimizer='Adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
model_12.fit(x_train, y_train,validation_data=(x_test,y_test), batch_size=1024, epochs=1000)

In [ ]:
preds2 = model_12.predict(test_data)
ans = []
for l in preds2:
    for n in range(len(l)):
        if l[n]==max(l):
            ans.append(n+1)
data ={"serial number": s_no, "cluster_number": ans }
df3 = pd.DataFrame(data=data)
df3.to_csv("submission_4.csv")